In [ ]:
import gc
import os
import cv2
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter, defaultdict
from PIL import Image

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix 

import torch
import torch.nn as nn
from torch.cuda.amp import autocast
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2 #np.array -> torch.tensor (B, 3, H, W)
import timm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
num_classes = 3
root_dir = '/kaggle/input/tw-food-101/tw_food_101/tw_food_101/'
csv_train_file = 'tw_food_101_train_with_folds.csv'
csv_test_file = 'tw_food_101_test_list.csv'
class_list = pd.read_csv("/kaggle/input/tw-food-101/tw_food_101_classes.csv", header=None).iloc[:,1].tolist()
label_dict = {cls: i for i, cls in enumerate(class_list)}

df = pd.read_csv(os.path.join(root_dir, csv_train_file))
test_df = pd.read_csv(os.path.join(root_dir, csv_test_file), header=None, names=['id','path_to_img'])

In [ ]:
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import os

class TWFoodDataset(Dataset):
    def __init__(self, root_dir, df, mode, transform=None):
        self.root = root_dir
        self.transform = transform
        self.mode = mode
        self.df = df
        self.img_path_list = df['path_to_img'].tolist()
        
        if 'class' in df.columns:
            self.labels = df['class'].tolist()
        else:
            self.labels = None   
    
    def __len__(self):
        return len(self.img_path_list)
        
    def __getitem__(self, idx):
        image_path = os.path.join(self.root, self.img_path_list[idx])
        
        # Try OpenCV first (faster)
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError("OpenCV couldn't read the image")
            # Convert BGR (OpenCV default) to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        # Fallback to PIL if OpenCV fails
        except:
            try:
                image = Image.open(image_path).convert('RGB')
                image = np.array(image)
            except Exception as e:
                raise Exception(f"Failed to load image {image_path} with both OpenCV and PIL: {str(e)}")
        
        # Apply transforms if provided
        if self.transform is not None:
            # Ensure image is in correct format for transforms
            transformed = self.transform(image=image)
            image = transformed['image']

        if self.mode == 'test':
            return image
        else:
            label = self.labels[idx]
            return image, torch.tensor(label).long()

In [ ]:
def get_transforms(image_size):
    transforms_train = A.Compose([
        A.Resize(image_size, image_size),
        A.ImageCompression(quality_lower=80, quality_upper=100, p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.5),
        A.Flip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.OneOf([
            A.MotionBlur(blur_limit=5),
            A.MedianBlur(blur_limit=5),
            A.GaussianBlur(blur_limit=5),
            A.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CoarseDropout(num_holes_range=(1,1), hole_height_range=(8, 32), hole_width_range=(8, 32), p=0.25),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    transforms_val = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

    return transforms_train, transforms_val

In [ ]:
def get_weighted_sampler(dataset):
    labels = [dataset.dataset[idx][2] for idx in dataset.indices]
    class_counts = np.bincount(labels, minlength=num_classes)
    class_weights = 1.0 / (class_counts + 1e-6)
    sample_weights = [class_weights[label] for label in labels]
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    return sampler

In [ ]:
class FoodModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.n_classes = num_classes
        self.backbone = timm.create_model(
            'tf_efficientnet_b8.ra_in1k',
            pretrained=True,
            num_classes=self.n_classes,
        )

    def forward(self, x):
        x = self.backbone(x)
        return x

In [ ]:
# Plotting function
def plot_fold_history(fold, history):
    actual_epochs = len(history['train_auc'])
    plt.figure(figsize=(15, 5))
    
    # Plot auc
    plt.subplot(1, 2, 1)
    plt.plot(range(1, actual_epochs + 1), history['train_auc'], '-o', label='Train AUC', color='skyblue')
    plt.plot(range(1, actual_epochs + 1), history['val_auc'], '-o', label='Val AUC', color='lightcoral')
    plt.scatter(history['best_val_auc_epoch'], history['best_val_auc'], s=200, color='lightcoral')
    plt.text(history['best_val_auc_epoch'], history['best_val_auc'], f'max {history["best_val_auc"]:.4f}', size=12)
    plt.xlabel('Epoch')
    plt.ylabel('auc')
    plt.title(f'Fold {fold + 1} Auc')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(range(1, actual_epochs + 1), history['train_loss'], '-o', label='Train Loss', color='skyblue')
    plt.plot(range(1, actual_epochs + 1), history['val_loss'], '-o', label='Val Loss', color='lightcoral')
    plt.scatter(history['best_val_loss_epoch'], history['best_val_loss'], s=200, color='lightcoral')
    plt.text(history['best_val_loss_epoch'], history['best_val_loss'], f'min {history["best_val_loss"]:.4f}', size=12)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Fold {fold + 1} Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class EarlyStopper:
    def __init__(self, min_delta=0, patience=1):
        self.min_delta = min_delta
        self.patience = patience
        self.max_val_auc = -float('inf')
        self.count = 0
        
    def early_stop(self, val_auc):
        if self.max_val_auc < val_auc:
            self.max_val_auc = val_auc
            self.count = 0
        elif self.max_val_auc > val_auc + self.min_delta:
            self.count += 1
            if self.count >= self.patience:
                return True
        return False
        
# Custom GradualWarmupSchedulerV2
class GradualWarmupSchedulerV2:
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.optimizer = optimizer
        self.multiplier = multiplier
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        self.last_epoch = -1
        self.base_lrs = [group['lr'] for group in optimizer.param_groups]
    
    def step(self, epoch=None):
        if epoch is None:
            self.last_epoch += 1
        else:
            self.last_epoch = epoch
        
        if self.last_epoch <= self.total_epoch:
            # Warmup phase
            for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
                param_group['lr'] = lr
        elif self.after_scheduler:
            # Transition to after_scheduler
            if not self.finished:
                self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                self.finished = True
            self.after_scheduler.step(self.last_epoch - self.total_epoch)
    
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [ ]:
def train_epoch(model, loader, optimizer, criterion, device):
    train_loss_meter = AverageMeter()
    model.train()
    
    PROBS = []
    TARGETS = []
    
    for img, label in loader:
        optimizer.zero_grad()
        inputs = img.to(device)
        targets = label.to(device)
        
        with autocast():
            logits = model(inputs)
            loss = criterion(logits, targets)
        
        loss.backward()
        optimizer.step()
        
        train_loss_meter.update(loss.item(), inputs.size(0))
        
        with torch.no_grad():
            probs = F.softmax(logits.float(), dim=1).cpu().numpy()
            PROBS.append(probs)
            TARGETS.append(targets.cpu().numpy())
    
    # Concatenate all predictions and targets
    PROBS = np.concatenate(PROBS)
    TARGETS = np.concatenate(TARGETS)

    if not np.allclose(PROBS.sum(axis=1), 1.0, atol=1e-5):
        print("PROBS not summing to 1!")
    if np.any(TARGETS < 0) or np.any(TARGETS >= 101):
        print(f"Invalid TARGETS values: {TARGETS}")

    # Compute AUC over entire epoch
    try:
        train_auc = roc_auc_score(y_true=TARGETS, y_score=PROBS, multi_class='ovr')
    except ValueError as e:
        print(f"Sample of PROBS: {PROBS[0]}, sum: {PROBS.sum(axis=1)}")
        print(f"Error: {e}")
        
    return train_loss_meter.avg, train_auc

In [ ]:
# Validation epoch
def val_epoch(model, loader, criterion, device):
    model.eval()
    val_loss_meter = AverageMeter()
    
    PROBS = []
    TARGETS = []
    
    with torch.no_grad():
        for img, label in loader:
            inputs = img.to(device)
            targets = label.to(device)
            
            with autocast():
                logits = model(inputs)
                loss = criterion(logits, targets)
            
            val_loss_meter.update(loss.item(), inputs.size(0))
            
            probs = F.softmax(logits.float(), dim=1).cpu().numpy()
            PROBS.append(probs)
            TARGETS.append(targets.cpu().numpy())
    
    PROBS = np.concatenate(PROBS)
    TARGETS = np.concatenate(TARGETS)
    
    try:
        val_auc = roc_auc_score(TARGETS, PROBS, multi_class='ovr')
    except ValueError as e:
        print(f"Val AUC failed: {e}, Unique targets: {np.unique(TARGETS)}, Probs shape: {PROBS.shape}")
        val_auc = 0.0
    
    return val_loss_meter.avg, val_auc

In [ ]:
def run(fold, df, root_dir, test_df, transforms_train, transforms_val, num_workers, n_epochs):
    train_df = df[df['fold'] != fold].reset_index(drop=True)
    val_df = df[df['fold'] == fold].reset_index(drop=True)

    # Datasets
    train_ds = TWFoodDataset(root_dir, train_df,'train', transform=transforms_train)
    val_ds = TWFoodDataset(root_dir, val_df,'train', transform=transforms_val)
    
    # Data loaders
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True, 
        pin_memory=True, 
        prefetch_factor=2
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False, 
        pin_memory=True, 
        prefetch_factor=2
    )

    # Model, optimizer, criterion
    model = FoodModel(num_classes=num_classes).to(device)
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Learning rate scheduler
    scheduler_cosine = CosineAnnealingWarmRestarts(optimizer, T_0=8)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=2, after_scheduler=scheduler_cosine)

    # History tracking
    history = {
        'train_loss': [], 'val_loss': [],
        'train_auc': [], 'val_auc': [],
        'learning_rates': [],
        'best_val_auc': 0, 'best_val_auc_epoch': 0,
        'best_val_loss': float('inf'), 'best_val_loss_epoch': 0
    }

    print(f"Fold {fold + 1}: =========================================")
    
    early_stopping_active = False
    # es = EarlyStopper(min_delta=1e-3, patience=3)
    
    for epoch in range(1, n_epochs + 1):
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)
        
        print(f"\nEP {epoch}/{n_epochs} (LR: {current_lr:.6f}):")
        train_loss, train_auc = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_auc = val_epoch(model, val_loader, criterion, device)
        
        print(f"Train AUC: {train_auc:.4f}")
        print(f"Val AUC: {val_auc:.4f}")
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_auc'].append(train_auc)
        history['val_auc'].append(val_auc)
        
        if val_auc > history['best_val_auc']:
            history['best_val_auc'] = val_auc
            history['best_val_auc_epoch'] = epoch
            torch.save(model.state_dict(), f'fold_{fold}_best_auc.pth')
            print(f"New best AUC! Model saved.")
        
        if val_loss < history['best_val_loss']:
            history['best_val_loss'] = val_loss
            history['best_val_loss_epoch'] = epoch
        
        scheduler_cosine.step()
        # if epoch == 2: scheduler_warmup.step()  # Bug workaround
            
        if epoch == scheduler_warmup.total_epoch:
            early_stopping_active = True
            # Reset early stopper to forget the potentially misleading high scores during warmup
            es = EarlyStopper(min_delta=1e-3, patience=2)
            print("Warmup complete. Early stopping now active.")
        
        # Only check early stopping if it's active
        if early_stopping_active:
            if es.early_stop(val_auc):
                print(f"Early stopping triggered at epoch {epoch}")
                break
        
    torch.save(model.state_dict(), f'fold_{fold}_final.pth')
    plot_fold_history(fold, history)
    
    # Compute OOF predictions after training
    model.load_state_dict(torch.load(f'fold_{fold}_best_auc.pth'))
    model.eval()
    oof_preds = []
    oof_targets = []
    with torch.no_grad():
        for img, label in val_loader:
            inputs = img.to(device)
            targets = label.to(device)
            logits = model(inputs)
            probs = F.softmax(logits, dim=1).cpu().numpy()
            oof_preds.append(probs)
            oof_targets.append(targets.cpu().numpy())
    
    oof_preds = np.concatenate(oof_preds)
    oof_targets = np.concatenate(oof_targets)
    
    # Compute test predictions
    test_ds = TWFoodDataset(root_dir, test_df, 'test', transform=transforms_val)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True, prefetch_factor=2)
    test_preds = []
    with torch.no_grad():
        for img in test_loader:
            preds = model(img.to(device))
            test_preds.append(F.softmax(preds, dim=1).cpu().numpy())
    test_preds = np.concatenate(test_preds)
    
    oof_names = val_df['id'].values
    oof_folds = np.full(len(oof_targets), fold)
    return oof_preds, oof_targets, oof_names, oof_folds, test_preds

In [ ]:
IMG_SIZE=256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
n_epochs = 15
num_workers = os.cpu_count() #check 
print(f"Num workers = {num_workers}")
folds=[0,1,2,3,4]
lr = 3e-4

oof_preds_all = []
oof_targets_all = []
test_preds_all = np.zeros((len(test_df), num_classes, len(folds)))

transforms_train, transforms_val = get_transforms(IMG_SIZE)

oof_preds_all = []
oof_targets_all = []
oof_names_all = []
oof_folds_all = []
test_preds_all = np.zeros((len(test_df), num_classes, len(folds)))

for fold in folds:
        oof_preds, oof_targets, oof_names, oof_folds, test_preds = run(fold, df, root_dir, test_df, transforms_train, transforms_val, num_workers,n_epochs=n_epochs)
        oof_preds_all.append(oof_preds)
        oof_targets_all.append(oof_targets)
        oof_names_all.append(oof_names)
        oof_folds_all.append(oof_folds)
        test_preds_all[:, :, fold] = test_preds
    
# Concatenate OOF data
oof_preds_all = np.concatenate(oof_preds_all)
oof_targets_all = np.concatenate(oof_targets_all)
oof_names_all = np.concatenate(oof_names_all)
oof_folds_all = np.concatenate(oof_folds_all)

# Compute overall OOF AUC
auc = roc_auc_score(oof_targets_all, oof_preds_all, multi_class='ovr')
print(f'Overall OOF AUC = {auc:.3f}')

# Save OOF to CSV
df_oof = pd.DataFrame({
    'image_name': oof_names_all,
    'target': oof_targets_all,
    'pred': oof_preds_all.argmax(axis=1),
    'fold': oof_folds_all
})
df_oof.to_csv('oof_8.csv', index=False)
print("OOF saved to 'oof_8.csv'")
print(df_oof.head())

# Average test predictions across 5 folds
test_preds_final = test_preds_all.mean(axis=2)  # [n_test, 101]